<hr>
<h1>Bullish Engulfing</h1>
<hr>

In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import talib as ta
import yfinance as yf
import datetime

In [26]:
data = yf.download(tickers='^NSEI', period='60d', interval='5m')
data.to_csv("test.csv")

[*********************100%***********************]  1 of 1 completed


In [27]:
data = pd.read_csv('/Users/tejasmadhukar/Documents/Vscode/Data_s/Probability project/test.csv')
# data = pd.read_csv('/Users/tejasmadhukar/Documents/Vscode/Data_s/Probability project/Data/NIFTY 50_5min.csv')
# data = data[data['Datetime'].dt.year==2021]
data['Datetime'] = pd.to_datetime(data['Datetime'])

In [28]:
# data = data.iloc[75:]
# data = data[data['Datetime'].dt.month==1]
data

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2022-10-28 09:15:00,17756.400391,17804.900391,17748.800781,17793.800781,17793.800781,0
1,2022-10-28 09:20:00,17793.900391,17803.099609,17786.650391,17802.050781,17802.050781,0
2,2022-10-28 09:25:00,17802.500000,17805.550781,17790.250000,17795.300781,17795.300781,0
3,2022-10-28 09:30:00,17795.699219,17812.650391,17785.750000,17808.250000,17808.250000,0
4,2022-10-28 09:35:00,17808.699219,17808.699219,17790.849609,17802.699219,17802.699219,0
...,...,...,...,...,...,...,...
4496,2023-01-20 15:10:00,18031.150391,18032.900391,18024.750000,18029.349609,18029.349609,0
4497,2023-01-20 15:15:00,18029.449219,18032.750000,18024.849609,18025.150391,18025.150391,0
4498,2023-01-20 15:20:00,18025.650391,18028.699219,18017.449219,18025.150391,18025.150391,0
4499,2023-01-20 15:25:00,18025.250000,18027.349609,18019.500000,18027.300781,18027.300781,0


In [29]:
def enter(data,j,sl,entry,target,arr,ana,count,sl_m,w_m):
    k = j+1
    while(k<len(data['Close'])):
        
        if data['Datetime'].dt.date.values[k] not in sl_m:
            sl_m[data['Datetime'].dt.date.values[k]] = 0
            
        if data['Datetime'].dt.date.values[k] not in w_m:
            w_m[data['Datetime'].dt.date.values[k]] = 0
            
        if(data['Datetime'].dt.day.values[k]>data['Datetime'].dt.day.values[j]):
            change = (entry-(data['High'].values[k-1]))/entry
            arr.append(change)
            ch_b = change>0
            if(change<0):
                sl_m[data['Datetime'].dt.date.values[k]] += 1
            else:
                w_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "{6} sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{7}, entry-time{4} , exit-time:{5}".format(sl,entry,data['Close'].values[k],target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],ch_b,data['Datetime'].dt.date.values[k])
            return k
        
        if(data['High'].values[k]>sl):
            change = (entry-sl)/entry
            arr.append(change)
            sl_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "-ve sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{6}, entry-time:{4} , exit-time:{5}".format(sl,entry,sl,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],data['Datetime'].dt.date.values[k])
            return k 
        
        if(data['Low'].values[k]<target):
            change = (entry-target)/entry
            arr.append(change)
            w_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "+ve sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{6}, entry-time:{4} , exit-time:{5}".format(sl,entry,target,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],data['Datetime'].dt.date.values[k])
            return k
        
        # if(data['Close'].values[k]<target):
        #     if(data['Close'].values[k+1]<target):
        #         target = entry - (4*(sl-entry))
        
        k += 1

In [30]:
def trade(data,i,arr,ana,count,sl_m,w_m):
    sl = 0.0
    sl_h = 0.0
    entry = 0.0
    target = 0.0
    j = i+1
    while(j<len(data['Close'])-1):
#         alter entering condition like if next candle opens above then only enter to optimise if()
        sl = min(data['Low'].values[j-1],data['Low'].values[j-2])
        entry = data['Open'].values[j] + ((data['Close'].values[j]-data['Open'].values[j])/2)
        target = entry - (1.5*(sl-entry))
        if(entry>sl):
            return j 
        return enter(data,j,sl,entry,target,arr,ana,count,sl_m,w_m)
        j += 1

In [31]:
def EMA(arr,data,ana,sl_m,w_m):
    i = 1
    count = 0
    while(i<len(data['Close'])-1):
        
        if data['Datetime'].dt.date.values[i] not in sl_m:
            sl_m[data['Datetime'].dt.date.values[i]] = 0
            
        if data['Datetime'].dt.date.values[i] not in w_m:
            w_m[data['Datetime'].dt.date.values[i]] = 0
            
        if(sl_m[data['Datetime'].dt.date.values[i]]>1):
            i += 1
            continue
            
        if(data['High'].values[i]>data['High'].values[i-1] and data['Low'].values[i]<data['Low'].values[i-1]):
            count += 1
            a = trade(data,i,arr,ana,count,sl_m,w_m)
            try:
                b = a-i+1
                i += b
                continue
            except:
                c = 1+1
        i += 1

In [32]:
arr = []
ana = {}
sl_m = {}
w_m = {}
EMA(arr,data,ana,sl_m,w_m)                